# 📓 Ingestion Module


In [ ]:
input_path = f"/lakehouse/base/parquet_{row_count}"
df = spark.read.format("parquet").load(input_path)

# Apply ingestion strategy
if strategy == "overwrite":
    df.write.mode("overwrite").saveAsTable("target_table")
elif strategy == "append":
    df.write.mode("append").saveAsTable("target_table")